In [1]:
import dynet as dy

In [2]:
pc = dy.ParameterCollection()
NUM_LAYERS=2
INPUT_DIM=50
HIDDEN_DIM=10
builder = dy.LSTMBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, pc)

In [3]:
builder

In [4]:
s0 = builder.initial_state()

In [5]:
s0

In [6]:
x1 = dy.vecInput(INPUT_DIM)

In [7]:
s1 = s0.add_input(x1)

In [8]:
s1

In [9]:
y1 = s1.output()

In [10]:
y1

expression 32/0

In [11]:
y1.npvalue()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
s2 = s1.add_input(x1)

In [15]:
s2

s0 -> s1 -> s2

In [16]:
y2 = s2.output()

In [17]:
y2

expression 62/0

In [18]:
s2.h()

(expression 47/0, expression 62/0)

In [19]:
for item in s2.h():
    print(item)

expression 47/0
expression 62/0


In [21]:
s1.h()

(expression 19/0, expression 32/0)

In [22]:
rnnbuilder=dy.SimpleRNNBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, pc)

In [23]:
# initialize a new graph, and a new sequence
rs0 = rnnbuilder.initial_state()

In [24]:
rs1 = rs0.add_input(x1)

In [25]:
ry1 = rs1.output()

`.s()`The internal state of the RNN

In [26]:
s1.s()

(expression 17/0, expression 30/0, expression 19/0, expression 32/0)

In [27]:
s0.s()

()

In [28]:
s2.s()

(expression 45/0, expression 60/0, expression 47/0, expression 62/0)

In [29]:
rnnh = rs1.h()
rnns = rs1.s()

In [31]:
print(rnnh, rnns, sep="\n")

(expression 70/0, expression 72/0)
(expression 70/0, expression 72/0)


In [32]:
lstm_h = s1.h()
lstm_s = s1.s()

In [33]:
print(lstm_h, lstm_s, sep="\n")

(expression 19/0, expression 32/0)
(expression 17/0, expression 30/0, expression 19/0, expression 32/0)


The LSTM has two extra state expressions (one for each hidden layer) before the outputs h.

In [35]:
s2.h(), s2.s()

((expression 47/0, expression 62/0),
 (expression 45/0, expression 60/0, expression 47/0, expression 62/0))

In [36]:
s2 = s1.add_input(x1)

In [37]:
s2.h(), s2.s()

((expression 87/0, expression 102/0),
 (expression 85/0, expression 100/0, expression 87/0, expression 102/0))

In [38]:
s3 = s2.add_input(x1)
s4 = s3.add_input(x1)

In [39]:
s5 = s3.add_input(x1)

we now have two different sequences:
`s0,s1,s2,s3,s4`
`s0,s1,s2,s3,s5`
the two sequences share parameters.

In [40]:
s5.prev() == s3

True

In [41]:
s4.prev() == s3

True

In [42]:
s6 = s3.prev().add_input(x1)

s0,s1,s2,s6

In [43]:
s6.h(), s6.s()

((expression 207/0, expression 222/0),
 (expression 205/0, expression 220/0, expression 207/0, expression 222/0))

In [44]:
state = rnnbuilder.initial_state()
xs = [x1,x1,x1]
states = state.add_inputs(xs)

In [45]:
states

In [46]:
outputs = [s.output() for s in states]
hs = [s.h() for s in states]
print(outputs, hs)

[expression 226/0, expression 230/0, expression 234/0] [(expression 224/0, expression 226/0), (expression 228/0, expression 230/0), (expression 232/0, expression 234/0)]


In [47]:
state = rnnbuilder.initial_state()
xs = [x1,x1,x1]
outputs = state.transduce(xs)
print(outputs)

[expression 238/0, expression 242/0, expression 246/0]


In [48]:
outputs[0].npvalue()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [49]:
import random
from collections import defaultdict
from itertools import count
import sys

In [53]:
LAYERS = 1
INPUT_DIM = 50
HIDDEN_DIM = 50

characters = list("abcdefghijklmnopqrstuvwxyz ")
characters.append("<EOS>")

int2char = characters
char2int = {c:i for i,c in enumerate(characters)}

VOCAB_SIZE = len(characters)

In [54]:
char2int

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 ' ': 26,
 '<EOS>': 27}

In [55]:
pc = dy.ParameterCollection()

srnn = dy.SimpleRNNBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, pc)
lstm = dy.LSTMBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, pc)

add parameters for the hidden->output part for both lstm and srnn

In [58]:
srnn = dy.SimpleRNNBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, pc)
lstm = dy.LSTMBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, pc)
params_lstm = {}
params_srnn = {}
for params in [params_lstm, params_srnn]:
    params["lookup"] = pc.add_lookup_parameters((VOCAB_SIZE, INPUT_DIM))
    params["R"] = pc.add_parameters((VOCAB_SIZE, HIDDEN_DIM))
    params["bias"] = pc.add_parameters((VOCAB_SIZE))

return compute loss of RNN for one sentence

In [78]:
def do_one_sentence(rnn, params, sentence):
    # setup the sentence
    dy.renew_cg()
    s0 = rnn.initial_state()

    R = params["R"]
    bias = params["bias"]
    lookup = params["lookup"]
    sentence = ["<EOS>"] + list(sentence) + ["<EOS>"]
    sentence = [char2int[c] for c in sentence]  # one hot represectation
    s = s0
    loss = []
    for char,next_char in zip(sentence,sentence[1:]):
        s = s.add_input(lookup[char])  # create an Expression from row char
#         print(s.output().npvalue().shape)
#         print(R.npvalue().shape)
        probs = dy.softmax(R*s.output() + bias)
        loss.append( -dy.log(dy.pick(probs,next_char)))
    loss = dy.esum(loss)
    return loss

generate from model:

In [79]:
def generate(rnn, params):
    def sample(probs):
        rnd = random.random()
        for i,p in enumerate(probs):
            rnd -= p
            if rnd <= 0: break
        return i

    # setup the sentence
    dy.renew_cg()
    s0 = rnn.initial_state()

    R = params["R"]
    bias = params["bias"]
    lookup = params["lookup"]

    s = s0.add_input(lookup[char2int["<EOS>"]])
    out=[]
    while True:
        probs = dy.softmax(R*s.output() + bias)
        probs = probs.vec_value()
        next_char = sample(probs)
        out.append(int2char[next_char])
        if out[-1] == "<EOS>": break
        s = s.add_input(lookup[next_char])
    return "".join(out[:-1]) # strip the <EOS>

train, and generate every 5 samples

In [80]:
def train(rnn, params, sentence):
    trainer = dy.SimpleSGDTrainer(pc)
    for i in range(200):
        loss = do_one_sentence(rnn, params, sentence)
        loss_value = loss.value()
        loss.backward()
        trainer.update()
        if i % 5 == 0:
            print("%.10f" % loss_value, end="\t")
            print(generate(rnn, params))

In [81]:
sentence = "a quick brown fox jumped over the lazy dog"
train(srnn, params_srnn, sentence)

0.0174368415	a quick brown fox jumped over the lazy dog
0.0172813181	a quick brown fox jumped over the lazy dog
0.0171286594	a quick brown fox jumped over the lazy dog
0.0169795789	a quick brown fox jumped over the lazy dog
0.0168315712	a quick brown fox jumped over the lazy dog
0.0166859496	a quick brown fox jumped over the lazy dog
0.0165432505	a quick brown fox jumped over the lazy dog
0.0164032970	a quick brown fox jumped over the lazy dog
0.0162657872	a quick brown fox jumped over the lazy dog
0.0161301866	a quick brown fox jumped over the lazy dog
0.0159962568	a quick brown fox jumped over the lazy dog
0.0158650093	a quick brown fox jumped over the lazy dog
0.0157360286	a quick brown fox jumped over the lazy dog
0.0156079428	a quick brown fox jumped over the lazy dog
0.0154845677	a quick brown fox jumped over the lazy dog
0.0153611340	a quick brown fox jumped over the lazy dog
0.0152393728	a quick brown fox jumped over the lazy dog
0.0151195750	a quick brown fox jumped over the l

In [82]:
sentence = "a quick brown fox jumped over the lazy dog"
train(lstm, params_lstm, sentence)

0.4231042266	a quick brown fox jumped over the lay dog
0.3977112472	a quick brown fox jumped over the lazy dog
0.3750352561	a quick brown fox juped oover the lazy dog
0.3546702266	a quicc brrown fox jumped over the lazy dog
0.3362885416	a quick brown fox jumped over the lazy dog
0.3196204901	a quick brown fox jumped over the lazy dog
0.3044435680	a quick brown fox jumped ove the lazy dog
0.2905706763	a quick brown fox jumped over the lazy dog
0.2778420150	a quick brown fox jumped over the lazy dog
0.2661255598	a quick brown fox jumped over the lazy dog
0.2553093135	a quick brown fox jumped over the lazy dog
0.2452945262	a quick brown fox jumped over the lazy og
0.2359956354	a quick brown fox jumped over the lazy dog
0.2273416668	a quick brown fox jumped over the lazy dog
0.2192698568	a quick brown fox jumped over the lazy dog
0.2117234468	a quick brown fox jumped over the lazy dog
0.2046546340	a quick brown fox jumped oo
0.1980202198	a quick brown fox jumped over the lazy dog
0.1917803